In [1]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import json
import re

In [2]:
ITEMS_COUNT = 100

In [4]:
middle_category_nums = ['001006', '001004', '001005', '001010', '001002', '001003',
                        '001001', '001011', '001013', '001008', '002022', '002001',
                        '002002', '002025', '002017', '002003', '002020', '002019',
                        '002023', '002018', '002004', '002008', '002007', '002024',
                        '002009', '002013', '002012', '002016', '002021', '002014',
                        '002006', '002015', '003002', '003007', '003008', '003004',
                        '003009', '003005', '003010', '003011', '003006', '002006',
                        '002007', '002008', '022001', '022002', '022003']

In [38]:
len(middle_category_nums)

47

In [5]:
# 사용자 에이전트
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

In [10]:
def get_response(url, headers):

    # GET 요청
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(0.5)

    return soup


In [33]:
def get_items(num):
    
    url = f'https://www.musinsa.com/categories/item/{num}?d_cat_cd={num}&brand=&list_kind=small&sort=sale_high&sub_sort=1d&page=1&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='
    
    product_links = []
    flag = 0
    page = 1
    while flag == 0:
        
        soup = get_response(url, headers)

        products = soup.select('a.img-block')
        
        for product in products:
            product_links.append(product['href'].replace('//', 'https://'))
            if len(product_links) == ITEMS_COUNT:
                flag = 1
                break

        page += 1
        url = f'https://www.musinsa.com/categories/item/{num}?d_cat_cd={num}&brand=&list_kind=small&sort=sale_high&sub_sort=1d&page={page}&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

    return product_links
        


In [ ]:
def get_item_info(item_url):
    soup = get_response(item_url, headers)
    info = soup.find_all('script', {'type':'text/javascript'})[15]
    info = info.string

    pattern = re.compile(r'window\.__MSS__\.product\.state = ({.*?});\s*$', re.DOTALL)
    match = pattern.search(info)
    info = match.group(1)
    
    return json.loads(info)

In [42]:
item_url = 'https://www.musinsa.com/app/goods/3175105'
soup = get_response(item_url, headers)

info = soup.find_all('script', {'type':'text/javascript'})[15]
info = info.string

pattern = re.compile(r'window\.__MSS__\.product\.state = ({.*?});\s*$', re.DOTALL)
match = pattern.search(info)
info = match.group(1)

info = json.loads(info)

In [69]:
info

{'goodsNo': 3175105,
 'goodsNm': '린넨 버튼 카라 반팔 니트_NND2ET1994',
 'goodsNmEng': '린넨 버튼 카라 반팔 니트_NND2ET1994',
 'thumbnailImageUrl': '/images/goods_img/20230323/3175105/3175105_16795555556281_500.jpg',
 'headDesc': None,
 'styleNo': 'NND2ET1994',
 'sex': ['남성'],
 'sexCode': 2,
 'brand': 'edition',
 'brandNm': '에디션',
 'brandNmEng': 'EDITION',
 'seasonYear': '2023',
 'season': '1',
 'couponDcPrice': 0,
 'comId': 'ssts4',
 'specialtyCodes': ['outlet'],
 'goodsEventType': 'N',
 'isRestictedUsePoint': True,
 'isClearance': True,
 'isPrePoint': False,
 'isGivenPoint': True,
 'isParallelImport': False,
 'deliveryDueType': 'NONE',
 'deliveryDuePeriod': 0,
 'isLimitedQuantity': False,
 'isLimitedTotalQuantity': False,
 'limitedMaxQuantity': 999,
 'limitedMinQuantity': 1,
 'isBuyForMember': False,
 'isAppGoods': False,
 'isOfflineGoods': False,
 'isFreeReturn': False,
 'giftCount': 0,
 'deliveryDueDay': None,
 'isSalePeriod': False,
 'isSale': False,
 'saleStartDate': None,
 'saleEndDate': None,
 'is

In [ ]:
def extract_needs_info(item_info):
    # 무신사 상품번호
    goodsNo = item_info['goodsNo']

    # 브랜드 명명
    brand = item_info['brand']

    # 품번
    styleNo = item_info['styleNo']

    # 무신사 판매가
    originPrice = item_info['goodsPrice']['originPrice']

    # 무신사 회원가
    memberPrice = item_info['goodsPrice']['minPrice']

    # 상품 이미지 : 리스트에 str로 저장. 필요하면 수정
    image_urls = []
    image_urls.append(item_info['thumbnailImageUrl'])
    goodsImages = item_info['goodsImages']
    for goodsImage in goodsImages:
        image_urls.append(goodsImage['imageUrl'])

    # 대분류
    categoryDepth1Title = item_info['category']['categoryDepth1Title']

    # 중분류
    categoryDepth2Title = item_info['category']['categoryDepth2Title']

    # 누적 판매

    # 좋아요 수

    # 나이

    # 성비




In [39]:
def main():
    all_product_links = []
    for middle_category_num in middle_category_nums:
        product_links = get_items(middle_category_num)
        print(len(product_links))
        all_product_links += product_links

    for item_url in all_product_links:
        item_info = get_item_info(item_url)
        

NameError: name 'all_product_links' is not defined

In [40]:
if __name__ == "__main__":
    main()

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
4700
